In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from perlin_noise import PerlinNoise

outp_dir = 'img/output_lattice_noise_images/'

%matplotlib inline

In [ ]:
def find_ft(img):
    ft = np.fft.fft(img)
    return np.fft.fftshift(ft)


def find_ift(ft):
    ift = np.fft.ifftshift(ft)
    return np.fft.ifft(ift)


def find_ft_2d(img):
    ft = np.fft.fft2(img)
    return np.fft.fftshift(ft)


def find_ift_2d(ft):
    ift = np.fft.ifftshift(ft)
    return np.fft.ifft2(ift)


def normalize_img(img):
    img_min = abs(np.min(img))
    img_max = abs(np.max(img))
    img_norm = 255 * (img + img_min) / (img_min + img_max)
    return img_norm.astype(int)


def normalize_to_one(img):
    img = np.array(img)
    img_min = abs(np.min(img))
    img_max = abs(np.max(img))
    img_norm = (img + img_min) / (img_min + img_max)
    return img_norm


def generate_single_octave_plane(width, grid_size, random_values=None, func=None):
    num_cells_x = width // grid_size
    
    if random_values is None:
        random_values = np.random.rand(num_cells_x + 1)

    noise_map = np.zeros(width)

    for x in range(width):
        cell_x = x // grid_size

        local_x = (x % grid_size) / grid_size

        left = random_values[cell_x]
        right = random_values[cell_x + 1]
        
        if func is None:
            smooth_x = local_x
        else:
            smooth_x = func(local_x)

        interp = left * (1 - smooth_x) + right * smooth_x

        noise_map[x] += interp

    return random_values, noise_map


def mesh_plane(x_min, x_max, x_step):
    x_lines = [0]

    for i in range(x_min, x_max, x_step):
        x_lines.append(x_lines[-1] + x_step)

    return x_lines


def qubic_interp(x):
    return x * x * (3 - 2 * x)


def fifth_order_interp(x):
    return 6 * x ** 5 - 15 * x ** 4 + 10 * x ** 3


def generate_value_noise(width, height, grid_size, num_octaves, persistence, func=None):
    noise_map = np.zeros((width, height))
    amplitude = 1

    for _ in range(num_octaves):
        noise_map += amplitude * generate_single_octave(width, height, grid_size, func=func)
        amplitude *= persistence
        grid_size = grid_size // 2

    # Normalize the values to [0, 1]
    noise_map = (noise_map - np.min(noise_map)) / (np.max(noise_map) - np.min(noise_map))
    return noise_map


def generate_single_octave(width, height, grid_size, random_values=None, func=None):
    num_cells_x = width // grid_size
    num_cells_y = height // grid_size
    
    if random_values is None:
        random_values = np.random.rand(num_cells_x + 1, num_cells_y + 1)

    noise_map = np.zeros((width, height))

    for y in range(height):
        for x in range(width):
            cell_x = x // grid_size
            cell_y = y // grid_size

            local_x = (x % grid_size) / grid_size
            local_y = (y % grid_size) / grid_size

            top_left = random_values[cell_x, cell_y]
            top_right = random_values[cell_x + 1, cell_y]
            bottom_left = random_values[cell_x, cell_y + 1]
            bottom_right = random_values[cell_x + 1, cell_y + 1]
            
            if func is None:
                smooth_x = local_x
                smooth_y = local_y
            else:
                smooth_x = func(local_x)
                smooth_y = func(local_y)

            interp_top = top_left * (1 - smooth_x) + top_right * smooth_x
            interp_bottom = bottom_left * (1 - smooth_x) + bottom_right * smooth_x

            noise_map[x, y] += interp_top * (1 - smooth_y) + interp_bottom * smooth_y
    return noise_map


def gen_img(x_size, y_size):
    xx = np.linspace(-x_size / 2, x_size / 2, x_size)
    yy = np.linspace(-y_size / 2, y_size / 2, y_size)
    whitenoise = np.random.normal(0, 1, (y_size, x_size))
    return whitenoise


def show_images(*images, vmin=0, vmax=255, x_fig_size=10, cmap='gray', y_fig_size=10, graphs_per_row=2):
    if len(images) == 1:
        plt.imshow(images[0], cmap='gray', vmin=0, vmax=255)
    else:
        row_num = ceil(len(images) / graphs_per_row)
        col_num = ceil(len(images) / row_num)
    
        f, axes = plt.subplots(row_num, col_num, sharey=True, figsize=(x_fig_size, y_fig_size))

        for ax, img in zip(axes.flatten(), images):
            ax.imshow(img, cmap='gray', vmin=0, vmax=255)

In [ ]:
pic_size = 128
mesh_size = 16

random_values = np.random.rand(pic_size // mesh_size + 1)

In [ ]:
rand_vals, lin_noise = generate_single_octave_plane(pic_size, mesh_size, random_values=random_values)
rand_vals, qubic_noise = generate_single_octave_plane(pic_size, mesh_size, random_values=random_values, func=qubic_interp)

rand_vals_x = mesh_plane(0, pic_size - 1, mesh_size)
noise_x = mesh_plane(0, pic_size - 1, 1)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(noise_x, lin_noise)
ax.plot(noise_x, qubic_noise)
ax.scatter(rand_vals_x, rand_vals, marker='.')

ax.set_xticks(rand_vals_x)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
# plt.savefig(outp_dir + 'types_of_interp.png')

In [ ]:
mesh_size = 32
x_mesh = mesh_plane(0, pic_size - 1, 16)

freq = mesh_size
pers = 0.707
octaves = 3
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []
random_values_oct = []


fig = plt.figure(figsize=(10, 10))
for i in range(octaves):
    rv = np.random.rand(pic_size // freq + 1)
    random_values_oct.append(rv)
    
    rand_vals, noise = generate_single_octave_plane(pic_size, freq, random_values=rv)
    noise = [val * amps[i] for val in noise]
    rand_vals = [val * amps[i] for val in rand_vals]
    rand_vals_x = mesh_plane(0, pic_size - 1, freq)
    harmonics.append(noise)
    freq = freq // 2
    
    ax = fig.add_subplot(3, 2, i + 1)
    ax.plot(noise)   
    ax.scatter(rand_vals_x, rand_vals, marker='.')

    ax.set_xticks(x_mesh)
    ax.set_xticks(noise_x, minor=True)
    ax.grid(which='both')

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, pic_size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
  

harmonics = np.sum(harmonics, axis=0)
if np.max(harmonics) > 1:
    harmonics = normalize_to_one(harmonics)
       
scatter_vals = []
for index in rand_vals_x:
    if index == pic_size:
        scatter_vals.append(harmonics[pic_size - 1])
    else:
        scatter_vals.append(harmonics[index])
      
    
ax = fig.add_subplot(3, 2, i + 2)
ax.plot(harmonics)
ax.scatter(rand_vals_x, scatter_vals, marker='.')

ax.set_xticks(x_mesh)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
# plt.savefig(outp_dir + 'lin_octave_sum.png')

In [ ]:
mesh_size = 32
x_mesh = mesh_plane(0, pic_size - 1, 16)

freq = mesh_size
pers = 0.707
octaves = 3
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []


fig = plt.figure(figsize=(10, 10))
for i in range(octaves):
    rand_vals, noise = generate_single_octave_plane(pic_size, freq, random_values=random_values_oct[i], func=qubic_interp)
    noise = [val * amps[i] for val in noise]
    rand_vals = [val * amps[i] for val in rand_vals]
    rand_vals_x = mesh_plane(0, pic_size - 1, freq)
    harmonics.append(noise)
    freq = freq // 2
    
    ax = fig.add_subplot(3, 2, i + 1)
    ax.plot(noise)   
    ax.scatter(rand_vals_x, rand_vals, marker='.')

    ax.set_xticks(x_mesh)
    ax.set_xticks(noise_x, minor=True)
    ax.grid(which='both')

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, pic_size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
  

harmonics = np.sum(harmonics, axis=0)
if np.max(harmonics) > 1:
    harmonics = normalize_to_one(harmonics)
       
scatter_vals = []
for index in rand_vals_x:
    if index == pic_size:
        scatter_vals.append(harmonics[pic_size - 1])
    else:
        scatter_vals.append(harmonics[index])
      
    
ax = fig.add_subplot(3, 2, i + 2)
ax.plot(harmonics)
ax.scatter(rand_vals_x, scatter_vals, marker='.')

ax.set_xticks(x_mesh)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
# plt.savefig(outp_dir + 'cubic_octave_sum.png')

In [ ]:
mesh_size = 32
x_mesh = mesh_plane(0, pic_size - 1, 16)

freq = mesh_size
pers = 0.707
octaves = 3
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []


fig = plt.figure(figsize=(10, 10))
for i in range(octaves):
    rand_vals, noise = generate_single_octave_plane(pic_size, freq, random_values=random_values_oct[i], func=fifth_order_interp)
    noise = [val * amps[i] for val in noise]
    rand_vals = [val * amps[i] for val in rand_vals]
    rand_vals_x = mesh_plane(0, pic_size - 1, freq)
    harmonics.append(noise)
    freq = freq // 2
    
    ax = fig.add_subplot(3, 2, i + 1)
    ax.plot(noise)   
    ax.scatter(rand_vals_x, rand_vals, marker='.')

    ax.set_xticks(x_mesh)
    ax.set_xticks(noise_x, minor=True)
    ax.grid(which='both')

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, pic_size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
  

harmonics = np.sum(harmonics, axis=0)
if np.max(harmonics) > 1:
    harmonics = normalize_to_one(harmonics)
       
scatter_vals = []
for index in rand_vals_x:
    if index == pic_size:
        scatter_vals.append(harmonics[pic_size - 1])
    else:
        scatter_vals.append(harmonics[index])
      
    
ax = fig.add_subplot(3, 2, i + 2)
ax.plot(harmonics)
ax.scatter(rand_vals_x, scatter_vals, marker='.')

ax.set_xticks(x_mesh)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
# plt.savefig(outp_dir + 'fifth_order_octave_sum.png')

### Spectrum

In [ ]:
mesh_size = 16
x_mesh = mesh_plane(0, pic_size - 1, 16)

freq = mesh_size
pers = 0.707
octaves = 3
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []


fig = plt.figure(figsize=(10, 10))
for i in range(octaves):
    rand_vals, noise = generate_single_octave_plane(pic_size, freq, func=qubic_interp)
    noise = [val * amps[i] for val in noise]
    rand_vals = [val * amps[i] for val in rand_vals]
    rand_vals_x = mesh_plane(0, pic_size - 1, freq)
    harmonics.append(noise)
    freq = freq // 2
    
    ax = fig.add_subplot(4, 2, i + 1)
    ax.plot(noise)   
    ax.scatter(rand_vals_x, rand_vals, marker='.')

    ax.set_xticks(x_mesh)
    ax.set_xticks(noise_x, minor=True)
    ax.grid(which='both')

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, pic_size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
  

harmonics = np.sum(harmonics, axis=0)
if np.max(harmonics) > 1:
    harmonics = normalize_to_one(harmonics)
       
scatter_vals = []
for index in rand_vals_x:
    if index == pic_size:
        scatter_vals.append(harmonics[pic_size - 1])
    else:
        scatter_vals.append(harmonics[index])
      
    
ax = fig.add_subplot(4, 2, i + 2)
ax.plot(harmonics)
ax.scatter(rand_vals_x, scatter_vals, marker='.')

ax.set_xticks(x_mesh)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)

In [ ]:
qubic_noise -= np.mean(qubic_noise)
ft = np.abs(find_ft(qubic_noise)) 

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(1, 2, 1)
ax.loglog(ft[pic_size // 2:])

f_curve = [1 / (x ** 0.5) for x in range(1, pic_size // 2)]
ax.loglog(f_curve)
ax.grid(which='both')
ax.set_ybound(10 ** -3, 100)
plt.xlabel('k')
plt.ylabel('АЧХ')


octaves = harmonics - np.mean(harmonics)
#if np.max(octaves) > 1:
#    octaves = normalize_to_one(octaves)
ft = np.abs(find_ft(octaves)) 

ax = fig.add_subplot(1, 2, 2)
ax.loglog(ft[pic_size // 2:])

f_curve = [1 / (x ** 0.5) for x in range(1, pic_size // 2)]
ax.loglog(f_curve)
ax.grid(which='both')
ax.set_ybound(10 ** -3, 100)
plt.xlabel('k')
plt.ylabel('АЧХ')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
# plt.savefig(outp_dir + 'noise_spectrum_total.png')

In [ ]:
N = 1000000
filter_ = [1 / (1 + i ** 0.5) for i in range(1, N + 1)]

vals = np.random.randn(N)
vals -= np.mean(vals)

fft = find_ft(vals) * filter_
p_noise = find_ift(fft)
ach = np.abs(fft)

plt.loglog(ach / np.max(ach))
plt.loglog(filter_)

In [ ]:
# Parameters
num_samples = 1024
sampling_rate = 1000  # Hz

# Generate pink noise using the Voss-McCartney algorithm
pink_noise = np.random.randn(num_samples)
pink_noise = np.cumsum(pink_noise)
pink_noise -= np.mean(pink_noise)  # Center the signal

# Calculate the power spectral density
pink_freqs = np.fft.fftfreq(num_samples, d=1/sampling_rate)
pink_spectrum = np.abs(np.fft.fft(pink_noise)) ** 2

# Calculate the 1/f spectrum
frequencies = np.fft.fftfreq(num_samples, d=1/sampling_rate)
frequencies = frequencies[1:num_samples//2]
one_over_f_spectrum = 1 / frequencies

# Create the plot
plt.figure(figsize=(10, 6))
plt.loglog(pink_freqs[:num_samples//2], pink_spectrum[:num_samples//2], label='Pink Noise')
plt.loglog(frequencies, one_over_f_spectrum, label='1/f Noise', linestyle='dashed')

plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.title('Pink Noise Spectrum vs. 1/f Noise Spectrum')
plt.legend()
plt.grid(True)
plt.show()

### 2D case

In [ ]:
width = 128
height = 128
grid_size = 16
persistence = 0.707

In [ ]:
noise_2d = generate_value_noise(width, height, grid_size, 1, persistence, func=qubic_interp)
noise_2d_oct = generate_value_noise(width, height, grid_size, 5, persistence, func=qubic_interp)


fig = plt.figure(figsize=(10, 4))
ax1 = fig.add_subplot(1, 2, 1)
im1 = ax1.imshow(noise_2d, cmap='gray', vmin=0, vmax=1)
# fig.colorbar(im, ax=ax1)
plt.xlabel('а')

ax = fig.add_subplot(1, 2, 2)
im = ax.imshow(noise_2d_oct, cmap='gray', vmin=0, vmax=1)
plt.xlabel('б')

cax = fig.add_axes([0.9, 0.11, 0.02, 0.77])  # Adjust these values as needed
cbar = plt.colorbar(im, cax=cax)

plt.subplots_adjust(wspace=0)
# plt.savefig(outp_dir + 'value_noise_2d_case.png')
plt.show()

### Spectrum

In [ ]:
noise_2d_sp = noise_2d - np.mean(noise_2d)
noise_2d_sp = find_ft_2d(noise_2d_sp)

noise_2d_oct_sp = noise_2d_oct - np.mean(noise_2d_oct)
noise_2d_oct_sp = find_ft_2d(noise_2d_oct_sp)

plt.imshow(np.log(np.abs(noise_2d_sp)), cmap='gray')
plt.show()

plt.imshow(np.log(np.abs(noise_2d_oct_sp)), cmap='gray')
plt.colorbar()

### Perlin Noise

In [ ]:
noise = PerlinNoise(octaves=3)
noise1 = PerlinNoise(octaves=6)
noise2 = PerlinNoise(octaves=12)
noise3 = PerlinNoise(octaves=24)
noise4 = PerlinNoise(octaves=48)

xpix, ypix = 128, 128

pic_single_oct = [[noise([i/xpix, j/ypix]) for j in range(xpix)] for i in range(ypix)]
pic_single_oct = normalize_to_one(pic_single_oct)
pic_oct = []

for i in range(xpix):
    row = []
    for j in range(ypix):
        noise_val = 0.707 * noise1([i/xpix, j/ypix])
        noise_val += 0.707 ** 2 * noise2([i/xpix, j/ypix])
        noise_val += 0.707 ** 3 * noise3([i/xpix, j/ypix])
        noise_val += 0.707 ** 4 * noise4([i/xpix, j/ypix])
        row.append(noise_val)
    pic_oct.append(row)
pic_oct = normalize_to_one(pic_oct)
    

fig = plt.figure(figsize=(10, 4))
ax1 = fig.add_subplot(1, 2, 1)
im1 = ax1.imshow(pic_single_oct, cmap='gray', vmin=0, vmax=1)
plt.xlabel('а')

ax = fig.add_subplot(1, 2, 2)
im = ax.imshow(pic_oct, cmap='gray', vmin=0, vmax=1)
plt.xlabel('б')

cax = fig.add_axes([0.9, 0.11, 0.02, 0.77])  # Adjust these values as needed
cbar = plt.colorbar(im, cax=cax)

plt.subplots_adjust(wspace=0)
# plt.savefig(outp_dir + 'perlin_noise_2d_case.png')
plt.show()

In [ ]:
w_noise = np.random.normal(0, 1, 100000)

plt.plot(w_noise)
plt.show()

plt.plot(np.abs(find_ft(w_noise)))
plt.show()